In [2]:
# import package
import pandas as pd
import numpy as np
from collections import defaultdict

# define the interested columns
INTERESTED_COLUMNS = {
    'mouse_ccds': ['gene_id', 'ccds_id', 'ccds_status', 'match_type'],
    'mouse_ccds_attributes': ['#ccds', 'gene_id', 'ccds_status', 'attribute'],
    'human_ccds': ['gene_id', 'ccds_id', 'ccds_status', 'match_type'],
    'human_ccds_attributes': ['#ccds', 'gene_id', 'ccds_status', 'attribute']
}

# Data Analyze on RefSeq Data

## Data Cleaning

In [3]:
# Load the mouse CCDS data
mouse_ccds = pd.read_csv("data/mouse_CCDS.current.txt", sep="\t", comment="/")
mouse_ccds_attributes = pd.read_csv("data/mouse_CCDS_attributes.current.txt", sep="\t", comment="/")

# Load the human CCDS data
human_ccds = pd.read_csv("data/human_CCDS_current.txt", sep="\t", comment="/")
human_ccds_attributes = pd.read_csv("data/human_CCDS_attributes.current.txt", sep="\t", comment="/")

mouse_ccds = mouse_ccds[INTERESTED_COLUMNS['mouse_ccds']]
mouse_ccds_attributes = mouse_ccds_attributes[INTERESTED_COLUMNS['mouse_ccds_attributes']]
human_ccds = human_ccds[INTERESTED_COLUMNS['human_ccds']]
human_ccds_attributes = human_ccds_attributes[INTERESTED_COLUMNS['human_ccds_attributes']]

# drop the duplicate rows
mouse_ccds.drop_duplicates(inplace=True)
human_ccds.drop_duplicates(inplace=True)
mouse_ccds_attributes.drop_duplicates(inplace=True)
human_ccds_attributes.drop_duplicates(inplace=True)

# making column name universal
columnNameMapping = {
    "#ccds": 'ccds_id',
    "match_type": 'ccds_match_type',
    "attribute": 'ccds_attribute'
}

mouse_ccds.rename(columns=columnNameMapping, inplace=True)
mouse_ccds_attributes.rename(columns=columnNameMapping, inplace=True)
human_ccds.rename(columns=columnNameMapping, inplace=True)
human_ccds_attributes.rename(columns=columnNameMapping, inplace=True)

## Difference Analysis
Compare whether the `(gene_id, ccds_id)` pair exist in both ccds table and ccds_attribute_table

In [4]:
# Case 1: Excluding ccds_status for mouse_ccds
# Convert rows of mouse_ccds (excluding ccds_status) into tuples and save them in a set
mouse_ccds_set_no_status = set(mouse_ccds[['gene_id', 'ccds_id']].drop_duplicates().itertuples(index=False, name=None))

# Convert rows of mouse_ccds_attributes (excluding ccds_status) into tuples and save them in a set
mouse_ccds_attributes_set_no_status = set(mouse_ccds_attributes[['gene_id', 'ccds_id']].drop_duplicates().itertuples(index=False, name=None))

# Find elements in mouse_ccds_set_no_status but not in mouse_ccds_attributes_set_no_status
only_in_mouse_ccds_no_status = mouse_ccds_set_no_status.difference(mouse_ccds_attributes_set_no_status)
print("Report for mouse_ccds_set but not in mouse_ccds_attributes_set:")
print(f"Count: {len(only_in_mouse_ccds_no_status)}")
print(f"Percentage: {len(only_in_mouse_ccds_no_status) / len(mouse_ccds) * 100:.2f}%")
print(f"Examples: {list(only_in_mouse_ccds_no_status)[:5]}")

# Find elements in mouse_ccds_attributes_set_no_status but not in mouse_ccds_set_no_status
only_in_mouse_ccds_attributes_no_status = mouse_ccds_attributes_set_no_status.difference(mouse_ccds_set_no_status)
print("\nReport for mouse_ccds_attributes_set but not in mouse_ccds_set:")
print(f"Count: {len(only_in_mouse_ccds_attributes_no_status)}")
print(f"Percentage: {len(only_in_mouse_ccds_attributes_no_status) / len(mouse_ccds_attributes) * 100:.2f}%")
print(f"Examples: {list(only_in_mouse_ccds_attributes_no_status)[:5]}")


# Case 2: Excluding ccds_status for human_ccds
# Convert rows of human_ccds (excluding ccds_status) into tuples and save them in a set
human_ccds_set_no_status = set(human_ccds[['gene_id', 'ccds_id']].drop_duplicates().itertuples(index=False, name=None))

# Convert rows of human_ccds_attributes (excluding ccds_status) into tuples and save them in a set
human_ccds_attributes_set_no_status = set(human_ccds_attributes[['gene_id', 'ccds_id']].drop_duplicates().itertuples(index=False, name=None))

# Find elements in human_ccds_set_no_status but not in human_ccds_attributes_set_no_status
only_in_human_ccds_no_status = human_ccds_set_no_status.difference(human_ccds_attributes_set_no_status)
print("\nReport for human_ccds_set but not in human_ccds_attributes_set:")
print(f"Count: {len(only_in_human_ccds_no_status)}")
print(f"Percentage: {len(only_in_human_ccds_no_status) / len(human_ccds) * 100:.2f}%")
print(f"Examples: {list(only_in_human_ccds_no_status)[:5]}")

# Find elements in human_ccds_attributes_set_no_status but not in human_ccds_set_no_status
only_in_human_ccds_attributes_no_status = human_ccds_attributes_set_no_status.difference(human_ccds_set_no_status)
print("\nReport for human_ccds_attributes_set but not in human_ccds_set:")
print(f"Count: {len(only_in_human_ccds_attributes_no_status)}")
print(f"Percentage: {len(only_in_human_ccds_attributes_no_status) / len(human_ccds_attributes) * 100:.2f}%")
print(f"Examples: {list(only_in_human_ccds_attributes_no_status)[:5]}")

Report for mouse_ccds_set but not in mouse_ccds_attributes_set:
Count: 26742
Percentage: 96.82%
Examples: [(74156, 'CCDS26678.1'), (404315, 'CCDS52592.1'), (20446, 'CCDS36379.1'), (15185, 'CCDS40845.1'), (19043, 'CCDS50199.1')]

Report for mouse_ccds_attributes_set but not in mouse_ccds_set:
Count: 4
Percentage: 0.44%
Examples: [(22022, 'CCDS89968.1'), (68644, 'CCDS90655.1'), (211535, 'CCDS90243.1'), (68644, 'CCDS90654.1')]

Report for human_ccds_set but not in human_ccds_attributes_set:
Count: 36342
Percentage: 96.22%
Examples: [(6039, 'CCDS9558.1'), (319101, 'CCDS8834.1'), (390195, 'CCDS31559.1'), (8193, 'CCDS46064.2'), (51030, 'CCDS42274.1')]

Report for human_ccds_attributes_set but not in human_ccds_set:
Count: 1
Percentage: 0.07%
Examples: [(117134596, 'CCDS93115.1')]


## Discrepancy Analysis
Validating the discrepancy of `ccds_status` in ccds table and ccds_attribute table. Ex. there is case when `ccds_status` in ccds table with the same `(gene_id, ccds_id)` pair is different from the `ccds_status` in ccds_attribute table.

In [5]:
# Iterate over rows in mouse_ccds_attributes
for _, attr_row in mouse_ccds_attributes.iterrows():
    ccds_id = attr_row['ccds_id']
    gene_id = attr_row['gene_id']
    
    # Skip rows where ccds_id or gene_id is missing
    if pd.isnull(ccds_id) or pd.isnull(gene_id):
        continue
    
    # Find the corresponding row in mouse_ccds
    matching_rows = mouse_ccds[(mouse_ccds['ccds_id'] == ccds_id) & (mouse_ccds['gene_id'] == gene_id)]
    
    # If a matching row exists, compare ccds_status
    if not matching_rows.empty:
        if matching_rows.shape[0] > 1:
            print(f"Multiple matching rows found for ccds_id: {ccds_id}, gene_id: {gene_id}")
            continue

        ccds_status_attr = attr_row['ccds_status']
        ccds_status_ccds = matching_rows.iloc[0]['ccds_status']
        
        if ccds_status_attr != ccds_status_ccds:
            print(f"Mismatch found for ccds_id: {ccds_id}, gene_id: {gene_id}")
            print(f"mouse_ccds_attributes ccds_status: {ccds_status_attr}")
            print(f"mouse_ccds ccds_status: {ccds_status_ccds}")

## Validation on one to many relationship

In [6]:
# Stack the two dataframes vertically
stacked_df = pd.concat([mouse_ccds[['gene_id', 'ccds_id']], mouse_ccds_attributes[['gene_id', 'ccds_id']]])

# Verify the one-to-many relationship by grouping by gene_id and counting the number of associated ccds_id
gene_to_ccds_count = stacked_df.groupby('gene_id')['ccds_id'].nunique().sort_values(ascending=False)
print("One-to-Many Relationship (gene_id to ccds_id):")
print(gene_to_ccds_count.head())  # Example counts

# Verify the one-to-many relationship by grouping by ccds_id and counting the number of associated gene_id
ccds_to_gene_count = stacked_df.groupby('ccds_id')['gene_id'].nunique().sort_values(ascending=False)
print("\nOne-to-Many Relationship (ccds_id to gene_id):")
print(ccds_to_gene_count.head())  # Example counts

One-to-Many Relationship (gene_id to ccds_id):
gene_id
21957     15
226025    14
11994     14
12916     14
16536     12
Name: ccds_id, dtype: int64

One-to-Many Relationship (ccds_id to gene_id):
ccds_id
CCDS14803.1    1
CCDS48997.1    1
CCDS48995.1    1
CCDS48994.1    1
CCDS48993.1    1
Name: gene_id, dtype: int64


- One gene_id has multiple ccds_id
- One ccds_id only has one gene_id associated with that

# Finding status categories

In [ ]:
human_ccds['ccds_status'].value_counts()
# human_ccds_attributes['ccds_status'].value_counts()

# loop through all the ccds_status in all tables
ccds_status = set()
for df in [mouse_ccds, mouse_ccds_attributes, human_ccds, human_ccds_attributes]:
    ccds_status.update(df['ccds_status'].unique())

ccds_status

{'Candidate',
 'Public',
 'Reviewed, update pending',
 'Reviewed, withdrawal pending',
 'Under review, update',
 'Under review, withdrawal',
 'Withdrawn',
 'Withdrawn, inconsistent annotation'}

In [15]:
mouse_ccds['ccds_match_type'].value_counts()

ccds_match_type
Identical    27516
Partial         65
Name: count, dtype: int64

In [ ]:
# loop through all the ccds_status in all tables
ccds_attribute = set()
for df in [mouse_ccds_attributes, human_ccds_attributes]:
    ccds_attribute.update(df['ccds_aattribute'].unique())

ccds_attribute

KeyError: 'ccds_aattribute'